# Computer Assignment 2 Problem 2

In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# grab the name of files in the directory 'chrom'
# excluding the file with 'html' extension
# and store the names in a list

chrom_files = glob.glob('chrom/*')
chrom_files = [x for x in chrom_files if 'html' not in x]
print(f'# of files: {len(chrom_files)}')

# files ending with a are training data
# files ending with b are testing data

# list of training data
train_files = [x for x in chrom_files if 'a' in x]
print(f'# of training files: {len(train_files)}')

# list of testing data
test_files = [x for x in chrom_files if 'b' in x]
print(f'# of testing files:  {len(test_files)}')

# of files: 44
# of training files: 22
# of testing files:  22


In [3]:
# function that reads the contents of a file
# and returns a list of lines
def read_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # for each line, take only the string after '\t' and before '\n'
    lines = [x.split('\t')[1].split('\n')[0] for x in lines]

    return lines


In [4]:
# read all the training data
training_data = [read_file(x) for x in train_files]
# flatten the list of lists
training_data = [item for sublist in training_data for item in sublist]
print(f'# of training data: {len(training_data)}')

# read all the testing data
test_data = [read_file(x) for x in test_files]
# flatten the list of lists
test_data = [item for sublist in test_data for item in sublist]
print(f'# of testing data: {len(test_data)}')

# of training data: 2200
# of testing data: 2200


# Levenshtein distance

In [5]:
def levenshtein_distance(s1, s2):
    # Create a matrix to store the distances between substrings of s1 and s2
    dp = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]

    # Initialize the first row and column of the matrix
    for i in range(len(s1) + 1):
        dp[i][0] = i
    for j in range(len(s2) + 1):
        dp[0][j] = j

    # Fill in the matrix using dynamic programming
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            cost = 0 if s1[i - 1] == s2[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,  # Deletion
                dp[i][j - 1] + 1,  # Insertion
                dp[i - 1][j - 1] + cost  # Substitution
            )

    # The final value in the bottom-right corner of the matrix is the Levenshtein distance
    return dp[len(s1)][len(s2)]

In [6]:
# Example usage:
string1 = "kitten"
string2 = "sitting"
distance = levenshtein_distance(string1, string2)
print(f"Levenshtein distance between '{string1}' and '{string2}' is {distance}")

Levenshtein distance between 'kitten' and 'sitting' is 3


# String grammar hard C-means

In [7]:
class sgHCM:
    # first, initial the class with k value
    # then, call the class with test sample and train data
    def __init__(self, k=3, max_iter=10, tol=1e-5):
        # number of clusters
        self.k = k
        # maximum iteration
        self.max_iter = max_iter
        # tolerance (epsilon)
        self.tol = tol
        # initialize the list of distances between old and new centroids very high
        self.Et = np.inf
        self.train_data = None
        self.centroids = None

    def __call__(self, test_sample):
        # compute the distance between test_sample and the centroids
        self.distances = self.compute_distance(test_sample, self.centroids)

        # find the index of the closest centroid
        closest_centroid = np.argmin(self.distances)

        # assign the test sample to the closest centroid
        self.predicted_label = closest_centroid

        return self.predicted_label, self.centroids[closest_centroid]
        
    def fit(self, train_data):
        self.train_data = train_data
        # randomly select k centroids from train_data (or prototype vectors)
        self.centroids = np.random.choice(train_data, size=self.k, replace=False)

        # initialize the list of clusters
        self.clusters = [[] for _ in range(self.k)]

        # initialize the list of old centroids
        self.old_centroids = None

        # initialize the list of new centroids
        self.new_centroids = None

        # initialize the list of distances between test sample and each train data
        self.distances = None

        # initialize the list of predicted labels
        self.predicted_label = None

        self.u_ik = None
        self.n_it = None

        for t in range(self.max_iter):
            print(f'\nIteration {t + 1}')
            # assign each train data to the closest centroid
            self.clusters, self.u_ik = self.assign_clusters(self.train_data)

            # n_it is the number of train samples in the i-th cluster
            self.n_it = np.sum(self.u_ik, axis=0)

            # update the centroids
            self.old_centroids = self.centroids
            self.new_centroids = self.update_centroids(self.train_data)
            
            _ = self.centroids_uniqueness(self.new_centroids)
            # print(f'New centroids: {self.new_centroids}')

            # compute the difference between old and new centroids
            self.Et = self.compute_terminal_measure(self.old_centroids, self.new_centroids)

            # if the difference is less than the tolerance, stop the iteration
            if self.Et < self.tol:
                break

            # otherwise, update the centroids and continue
            self.centroids = self.new_centroids

        print(f'Final centroids: {self.centroids}')
        # return the list of clusters
        return self.clusters

    def assign_clusters(self, train_data):
        # compute the distance between the train data and each centroid
        # initialize the list of clusters
        clusters = [[] for _ in range(self.k)]

        # for each train data
        for i, train_sample in enumerate(train_data):
            # compute the distance between test_sample and the centroid
            self.distances = self.compute_distance(train_sample, self.centroids)

            # find the index of the closest centroid
            closest_centroid = np.argmin(self.distances)

            # assign the train sample to the closest centroid
            clusters[closest_centroid].append(i)

        # create the u_ik matrix, which is a matrix of 0s and 1s
        # u_ik[i][j] = 1 if the i-th train sample belongs to the j-th cluster
        # u_ik[i][j] = 0 otherwise
        u_ik = np.zeros((len(train_data), self.k))
        for i, cluster in enumerate(clusters):
            for j in cluster:
                u_ik[j][i] = 1

        self.n_it = np.sum(u_ik, axis=0)

        return clusters, u_ik
    
    def update_centroids(self, train_data):
        # compute the distance between each sample and other samples in the same cluster
        # the sample with the minimum sum of the distances to other samples in the same cluster is the new centroid for that cluster  

        # initialize the list of new centroids
        new_centroids = []

        # for each cluster
        for i in range(len(self.clusters)):
            # c_ij is a list of the sum of the distances to other samples in the same cluster
            c_ij = []

            for j in range(len(self.clusters[i])):
                # compute the distance between the sample and other samples in the same cluster excluding itself
                # list of samples excluding the sample itself
                samples = [x for x in self.clusters[i] if x != j]

                # take train_data at the index of samples
                train = np.array(train_data.copy())
                other_samples = train[samples]

                distances = self.compute_distance(train_data[j], other_samples)
                
                c_ij.append(np.sum(distances/self.n_it[i], axis=0))

            # find the index of the sample with the minimum sum 
            # of the distances to other samples in the same cluster
            alpha_q = np.argmin(c_ij)

            # add the new centroid to the list
            new_centroids.append(train_data[alpha_q])

        return new_centroids
    
    def centroids_uniqueness(self, centroids):
        # check if the new centroids are unique
        # return True if the new centroids are unique
        # return False if the new centroids are not unique
        # print(f'Checking centroids uniqueness...')
        for i in range(len(centroids)):
            for j in range(len(centroids)):
                if i != j:
                    if centroids[i] == centroids[j]:
                        print(f'Found centroids not unique')
                        # redraw the centroids
                        self.centroids = np.random.choice(self.train_data, size=self.k, replace=False)
                        return False
        return True
    
    def levenshtein_distance(self, s1, s2):
        # Create a matrix to store the distances between substrings of s1 and s2
        dp = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]

        # Initialize the first row and column of the matrix
        for i in range(len(s1) + 1):
            dp[i][0] = i
        for j in range(len(s2) + 1):
            dp[0][j] = j

        # Fill in the matrix using dynamic programming
        for i in range(1, len(s1) + 1):
            for j in range(1, len(s2) + 1):
                cost = 0 if s1[i - 1] == s2[j - 1] else 1
                dp[i][j] = min(
                    dp[i - 1][j] + 1,  # Deletion
                    dp[i][j - 1] + 1,  # Insertion
                    dp[i - 1][j - 1] + cost  # Substitution
                )

        # The final value in the bottom-right corner of the matrix is the Levenshtein distance
        return dp[len(s1)][len(s2)]

    def compute_distance(self, test_sample, train_data):
        # compute the distance between test_sample and each train data
        # return a list of distances
        distances = []
        for train_sample in train_data:
            distance = self.levenshtein_distance(test_sample, train_sample)
            distances.append(distance)
        return distances
    
    def compute_terminal_measure(self, old_V, new_V):
        # compute the levenstein distance between old and new centroids
        # return the sum of the levenstein distances
        Et = []
        for i in range(len(old_V)):
            Et.append(self.levenshtein_distance(old_V[i], new_V[i]))
    
        Et = np.array(Et)
        Et = np.sum(Et)
        print(f'Terminal measure: Et = {Et}')

        # return the tolerance
        return Et
    
# perform 10-fold cross validation on sgHCM classifier
def cross_validation(train_data, test_data, k_num = 2, fold=10):
    # shuffle data before cross validation
    np.random.shuffle(train_data)
    np.random.shuffle(test_data)
    fold_size = len(train_data) // fold

    for i in range(fold):
        # take one fold as training data
        data_train_fold = train_data[i * fold_size: (i + 1) * fold_size]
        data_test_fold = test_data[i * fold_size: (i + 1) * fold_size]

        ############################ sgHCM classifier ##########################
        print(f"\nFold {i+1}")
        classifier = sgHCM(k=k_num)
        classifier.fit(data_train_fold)

        # print('\nPredicting the test dataset...')
        print(f'Predictions: (test sample, predicted label, predicted centroid)')
        predictions = []
        # test the first 5 samples
        for test_sample in data_test_fold[:5]:
            predicted_label, predicted_centroids = classifier(test_sample)
            print(f'{test_sample}, {predicted_label}, {predicted_centroids}')
            predictions.append([test_sample, predicted_label, predicted_centroids])
     
        print('\n+++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')


In [8]:
# vary k from 2 to 5
for k in range(2, 6):
    print(f'\n\nk = {k}')
    cross_validation(training_data, test_data, k_num = k)



k = 2

Fold 1

Iteration 1


Terminal measure: Et = 27

Iteration 2
Terminal measure: Et = 0
Final centroids: ['A====A==a====E====e=====A==a=====a', 'A=A=a=====E====b====B===b===B===a==A======c====B=====d=====a']
Predictions: (test sample, predicted label, predicted centroid)
A=A=a=AaAa===E====b=A=b====B==d===Aa=a, 0, A====A==a====E====e=====A==a=====a
A===B======a==A===a==A==b====A==a=A=a=====A====a=Aa===C==a==A==a===C===c===A==b=A==b==A=a=a, 1, A=A=a=====E====b====B===b===B===a==A======c====B=====d=====a
A=C==a=C======d====C==a==A=a=Aa=A=a=A====c======D==========d==A=b=a, 1, A=A=a=====E====b====B===b===B===a==A======c====B=====d=====a
A=A=a==A==a====C=b===D===d=====A=====b===A=a=a, 0, A====A==a====E====e=====A==a=====a
A==Aa===C===b==A====a=AA=a===C=====a=A===d===A=a=A==b==Aa=a, 1, A=A=a=====E====b====B===b===B===a==A======c====B=====d=====a

Fold 2

Iteration 1
Terminal measure: Et = 26

Iteration 2
Terminal measure: Et = 0
Final centroids: ['A======B===a===A==b========E====d============a=a', 'A===A===a====E===

KeyboardInterrupt: 